# example krx

정보데이터시스템 : http://data.krx.co.kr/contents/MDC/MAIN/main/index.cmd

## json 생성
```
{
    "00593":"삼성전자"
}
```

In [5]:
import pandas as pd
import io
import requests as r
from datetime import datetime
import json

generate_otp_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
# today = datetime.today().strftime('%Y%m%d')
otp_data = {
    'mktId': 'ALL',
    'trdDd': '20210507',
    'share': 1,
    'money': 1,
    'csvxls_isNo' : False,
    'name': 'fileDown',
    'url' : 'dbms/MDC/STAT/standard/MDCSTAT01501'
}

otp = r.post(generate_otp_url, data = otp_data)
code = otp.content

down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
referer = {'referer': generate_otp_url,
          'user-agent': 'User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36'}
data = {'code': code}
down_sector_ks = r.post(down_url, data=data, headers = referer)
decoded_content = down_sector_ks.content.decode('cp949')
df = pd.read_csv(io.StringIO(decoded_content))
df = df[['종목코드', '종목명']]
code_dict = dict(zip(df['종목코드'], df['종목명']))

with open("example_code.json", "w") as w:
    json.dump(code_dict, w, ensure_ascii=False)

In [7]:
import pandas as pd
import glob
import os

list_of_files = glob.glob('/Users/henry/Downloads/*.csv')
latest_file = max(list_of_files, key=os.path.getctime)
df = pd.read_csv(latest_file, encoding="cp949")

df = df[['종목코드', '종목명']]
code_dict = dict(zip(df['종목코드'], df['종목명']))

with open("example_code.json", "w") as w:
    json.dump(code_dict, w, ensure_ascii=False)